In [2]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 33.5 MB/s eta 0:00:00


In [3]:
import pulp as pl

# Create a linear programming problem
model = pl.LpProblem("Production_Planning", pl.LpMaximize)

# Products
products = ["Product_A", "Product_B", "Product_C"]
profit = {"Product_A": 10, "Product_B": 12, "Product_C": 14}  # profit per unit
demand = {"Product_A": 100, "Product_B": 80, "Product_C": 50}  # max demand

# Resources
resources = ["Labor", "Material_X", "Material_Y"]
available = {"Labor": 480, "Material_X": 320, "Material_Y": 250}  # available hours/units

# Resource usage per unit of product
usage = {
    ("Product_A", "Labor"): 5,
    ("Product_B", "Labor"): 4,
    ("Product_C", "Labor"): 6,
    ("Product_A", "Material_X"): 3,
    ("Product_B", "Material_X"): 5,
    ("Product_C", "Material_X"): 2,
    ("Product_A", "Material_Y"): 2,
    ("Product_B", "Material_Y"): 1,
    ("Product_C", "Material_Y"): 4
}

In [4]:
# Decision variables - how many of each product to make
production = {p: pl.LpVariable(f"Produce_{p}", lowBound=0, cat='Integer') for p in products}

# Objective function: maximize profit
model += pl.lpSum([profit[p] * production[p] for p in products]), "Total_Profit"

# Resource constraints
for r in resources:
    model += pl.lpSum([usage[(p, r)] * production[p] for p in products]) <= available[r], f"Total_{r}"

# Demand constraints
for p in products:
    model += production[p] <= demand[p], f"Demand_{p}"

# Solve the model
model.solve()

1

In [5]:

# Print the results
print(f"Status: {pl.LpStatus[model.status]}")
print("\nOptimal Production Plan:")
for p in products:
    print(f"{p}: {production[p].value()} units")

print(f"\nTotal Profit: ${pl.value(model.objective)}")

# Resource usage
print("\nResource Usage:")
for r in resources:
    total_used = sum(usage[(p, r)] * production[p].value() for p in products)
    print(f"{r}: {total_used} / {available[r]} ({total_used/available[r]*100:.1f}%)")

Status: Optimal

Optimal Production Plan:
Product_A: 0.0 units
Product_B: 44.0 units
Product_C: 50.0 units

Total Profit: $1228.0

Resource Usage:
Labor: 476.0 / 480 (99.2%)
Material_X: 320.0 / 320 (100.0%)
Material_Y: 244.0 / 250 (97.6%)
